# Analysis

Once you have reconstructed one or more faces from videos, you can analyze them in similar ways as you would analyze electrophysiological (EEG/MEG) or fMRI data.Each vertex in the reconstructed data represents a signal, which varies across time. At each time point, the value of the signal represents the _location_ of the vertices; the values across time points thus represents variation in location, i.e., _movement_. And we can try to quantify facial movements just like we try to analyze brain signals!

Although there are many different kinds of analyses that you could do on this data (e.g., regression analyses, connectivity analyses, inter-personal synchrony, "decoding" analyses), here we showcase a very simple one: averaging an epoched signal (which is similar to ERP analysis in EEG/MEG and spike-triggered averaging).

N.B.: the analysis/epoching part of Medusa is very much work in progress still.

## Dataset

For this tutorial, we created a dummy dataset derived from this YouTube video, created by Andzej Gavriss:

In [ ]:
from IPython.display import YouTubeVideo, Video
YouTubeVideo('MTWkfpa-jJw')

This video contains short clips of different people with different facial expressions (e.g., smiling, frowning, crying). We split up this video into subvideos containing only a single person and single expression, which are shipped with Medusa:

In [ ]:
import medusa
from pathlib import Path

medusa_path = Path(medusa.__file__).parent
videos = sorted(list(medusa_path.glob('data/example_video_dataset/*.mp4')))
videos

For example, `smile06.mp4` shows actor 6 smiling:

In [ ]:
Video(videos[-2], embed=True, width=512)

And `anger04.mp4` shows actor 2 with an angry expression:

In [ ]:
Video(videos[1], embed=True, width=512)

The full list of videos and annotations, including expression onsets and offsets, are listed in the following file. (Note that for this tutorial we'll limit ourselves to the smiling videos.)

In [ ]:
import pandas as pd
annotations = pd.read_csv(medusa_path / 'data/example_video_dataset/info.tsv', sep='\t')
annotations = annotations.query("expression == 'smile'")

annotations

We'll delve into the annatations more later in this tutorial. 

We already reconstructed the videos (using the EMOCA model), which constitutes the data we'll work with:

In [ ]:
data_files = sorted(list(medusa_path.glob('data/example_video_dataset/smile*.h5')))
data_files

Let's take a look at the reconstruction from the previously shown smiling video:

In [ ]:
from medusa.render import VideoRenderer
from medusa.containers import Data4D

renderer = VideoRenderer(loglevel='WARNING')
data = Data4D.load(data_files[-2])
renderer.render('./viz/smile02_recon.mp4', data)

Video('./viz/smile02_recon.mp4', embed=True, width=512)

One of the most simple analyses we could do is to "average" the reconstructions from the different actors. To average the reconstructions, we first need to create epochs of the different videos, which effectively align the data so that they can be averaged. Like ERPs, we can extract the "activity" (i.e., facial movement) within a particular period time-locked to an event. For example, we could take the onset of the expression (which is annotated in the annotations file) and extract a window of -0.1 seconds to +2 seconds after the onset. Alternatively, we could simply extract all activity between the onset and peak (annotated as 'offset'). However, because some expressions take longer than others, we should then interpolate a fixed number of values in this window (e.g., 50). 

While it may sound simple, this epoching operation is in fact quite cumbersome. That's why we created an `EpochsArray` class, which stores the epoched data, and crucially, contains a classmethod (`from_4D`) to create an `EpochsArray` object from a list of `Data4D` files:

In [ ]:
from medusa.epoch import EpochsArray
EpochsArray.from_4D?

So to create an `EpochsArray` with the regularly interpolated activity between onset and offset for each video, we can do the following:

In [ ]:
epochs = EpochsArray.from_4D(data_files, annotations, anchor='span', T=50)

Crucially, the `EpochsArray` object (in "aligned" space) contains two sets of epoched data: `v_epochs` and `params_epochs`, representing the epoched vertices and the epoched global movement parameters respectively:

In [ ]:
# N x T x V x 3
print(epochs.v_epochs.shape)

# N x T x 12 (global mov. parameters)
print(epochs.params_epochs.shape)

With this epoched data, we can do any type of analysis you would do on EEG/MEG data, like fancy regression-based encoding analyses or classification-based decoding analyses. But as said before, the easiest type of analysis is a simple ERP-average. (Which is, of course, just an encoding/regression model with just an intercept as a predictor.) 

However, instead of averaging the raw values (i.e., absolute location of the vertices), you can consider only averaging the (relative) movement across reconstructions. This is the same as "baseline normalization" in electrophysiological ERP analyses. This can be done as follows:

In [ ]:
# Compute "grand mean" (average over recons of first timepoint/baseline) 
grand_mean = epochs.v_epochs[:, 0, :, :].mean(axis=0)[None, None, ...]

# Compute deviations from neutral (i.e., timepoint 0)
deviations = epochs.v_epochs - epochs.v_epochs[:, 0, None, :, :]

# Set epochs to deviations from neutral + grand mean
epochs.v_epochs = deviations + grand_mean

To average the epochs and convert it into a new `Data4D` object (for easy rendering), we can use the `to_4D` method:

In [ ]:
data_example = Data4D.load(path=data_files[-1])
data_example.to_local()
data_mean = epochs.to_4D(agg='mean', tris=data_example.tris, cam_mat=data_example.cam_mat)

# T x V x 3
print(data_mean.v.shape)

Finally, we can visualize the average "smile", or "smile ERP" (with the deviations relative to neutral as vertex colors):

In [ ]:
from medusa.render import Overlay

dev = (data_mean.v - data_mean.v[0, None, ...])
overlay = Overlay(dev, v0=data_mean.v[0], tris=data_mean.tris)
overlay = overlay.to_rgb()

renderer = VideoRenderer(loglevel='WARNING')
renderer.render('./viz/smile_erp.mp4', data_mean, viewport=(512, 512), fps=10, overlay=overlay)
Video('./viz/smile_erp.mp4', embed=True)

Note that we can go way beyond computing the average and perform statistics (e.g., compute $t$-values for each vertex of the amount of movement relative to the neutral frame), which will (hopefully) be implemented in the future.